In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score

from aijack.attack import MI_FACE
from aijack.utils import DataSet
from aijack.defense import GeneralMomentAccountant, PrivacyManager

from opacus import PrivacyEngine
# INPUT PATHS:
BASE = "data/"

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fla = nn.Flatten()
        self.fc = nn.Linear(112 * 92, 40)

    def forward(self, x):
        x = self.fla(x)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
if True:
    imgs = []
    labels = []
    for i in range(1, 41):
        for j in range(1, 11):
            img = cv2.imread(BASE + f"s{i}/{j}.pgm", 0)
            imgs.append(img)
            labels.append(i - 1)

    X = np.stack(imgs)
    y = np.array(labels)

    # ToTensor：画像のグレースケール化（RGBの0~255を0~1の範囲に正規化）、Normalize：Z値化（RGBの平均と標準偏差を0.5で決め打ちして正規化）
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
    )
    trainset = DataSet(X, y, transform=transform)

In [ ]:
lot_size = 40
iterations=10
sigma=0.5

net = Net()
criterion = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
        trainset,
        batch_size=lot_size,
        num_workers=0,
        pin_memory=True,
    )

optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private(
                module=net,
                optimizer=optimizer,
                data_loader=train_loader,
                noise_multiplier=sigma,
                max_grad_norm=1,
            )

In [ ]:
for epoch in range(iterations):  # loop over the dataset multiple times
        running_loss = 0
        data_size = 0
        for data in train_loader:
            X_batch, y_batch = data
            optimizer.zero_grad()
            loss = criterion(net(X_batch), y_batch.to(torch.int64))
            loss.backward()
            running_loss += loss.item()
            data_size += X_batch.shape[0]
            optimizer.step()
        print(f"epoch {epoch}: loss is {running_loss/data_size}")
        
        
        epsilon, best_alpha = privacy_engine.accountant.get_privacy_spent(
            delta=1e-5
        )
        print(epsilon, best_alpha)

In [ ]:
if True:
    in_preds = []
    in_label = []
    with torch.no_grad():
        for data in train_loader:
            inputs, labels = data
            outputs = net(inputs)
            in_preds.append(outputs)
            in_label.append(labels)
        in_preds = torch.cat(in_preds)
        in_label = torch.cat(in_label)
    print(accuracy_score(np.array(torch.argmax(in_preds, axis=1)), np.array(in_label)))

In [ ]:
privacy_engine.accountant.steps